In [6]:
import collections
import json
import re

import networkx as nx
import numpy as np
import pandas as pd
import requests
import scipy.sparse
import tqdm
import xswap

import analysis

%matplotlib inline

In [2]:
# Download and save TFTG relationships
tftg_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5795274/bin/12915_2017_469_MOESM5_ESM.gmt'
tftg_path = '../data/tftg_raw.gmt'

# with open(tftg_path, 'wb') as f:
#     res = requests.get(tftg_url)
#     f.write(res.content)

In [57]:
# Construct a DataFrame of TF-TG relationships
records = list()
with open(tftg_path, 'r') as f:
    for line in f.readlines():
        groups = line.split('\t')
        match = re.search('(?<=:\ )([A-Z0-9\-\_]+)(\()([0-9]+)', groups[1])
        if match:
            tf_name, _, tf_id = match.groups()
        else:
            tf_name, _, tf_id = re.search('([A-Z0-9\-]+)(\_)([0-9]+)$', groups[1]).groups()
            
        match = re.search('.+(?=\ Transcriptional Targets.+)', groups[1])
        if match:
            method = match.group().lower()
        else:
            method = re.search('.+(?=\ TFTG.+)', groups[1]).group().lower()
            
        for gene in groups[2:]:
            records.append(
                (tf_id, tf_name, method, gene.strip())
            )

np.random.seed(0)
df = (
    pd.DataFrame
    .from_records(records, columns=['tf_id', 'tf', 'method', 'gene'])
)

df.head()

,tf_id,tf,method,gene
0,10009,ZBTB33,low throughtput,CDKN2A
1,10009,ZBTB33,low throughtput,MMP7
2,10009,ZBTB33,low throughtput,S100A4
3,10009,ZBTB33,low throughtput,WNT11
4,10009,ZBTB33,low throughtput,GP1BA


## ISSUE:

There are way too many unique nodes in the full network. Unfortunately, using strongly connected components gives only 255 nodes, which is far too few. So either we need to select the first few connected components until there are a certain number of nodes (O(10^3)), or look for another network. But this is a really good network for this purpose.

In [61]:
edges = list(map(
    tuple, df.filter(items=['tf', 'gene']).values
))
original_edges = set(edges)

In [67]:
dg = nx.from_edgelist(edges, create_using=nx.DiGraph())

In [71]:
len(dg.nodes)

17167

In [68]:
dgc = max(nx.strongly_connected_component_subgraphs(dg), key=len)

In [73]:
dgc.number_of_edges()

3120

In [74]:
dgc.number_of_nodes()

255

In [58]:
# Select the largest connected component of the graph
g = nx.from_edgelist(edges)
gc = max(nx.connected_component_subgraphs(g), key=len)
connected_edges = list(map(tuple, gc.edges))

connected_edges = (
    [edge for edge in connected_edges if edge in original_edges]
    + [tuple(sorted(edge)) for edge in connected_edges if edge not in original_edges]
)

In [60]:
len(set(edge[0] for edge in edges)), len(set(edge[0] for edge in connected_edges))

(384, 8485)

In [52]:
# Map edges to unique integers (for XSwap)
mapped_edges, mapping, _ = xswap.preprocessing.map_str_edges(connected_edges, bipartite=False)
reversed_mapping = {v: k for k, v in mapping.items()}

# Construct a matrix (for feature computation)
mat = analysis.edges_to_matrix(mapped_edges)

# Create source, target degree matrices
degree = np.repeat(mat.sum(axis=1), mat.shape[1], axis=1) \
       + np.repeat(mat.sum(axis=0), mat.shape[0], axis=0)

In [ ]:
# Compute features on unpermuted network
feature_mats = {
    'prior_empirical': np.zeros(mat.shape),
    
    'rwr': analysis.invertible_rwr(mat.toarray(), 0.25),
    'mean_rwr': np.zeros(mat.shape),
    'p_rwr': np.zeros(mat.shape),
    
    'jaccard': (mat@mat.T) / (degree - mat@mat.T),
    'mean_jaccard': np.zeros(mat.shape),
    'p_jaccard': np.zeros(mat.shape),
}

# Compute RWR p-value
n_perms = 100
perm_edges = mapped_edges.copy()
for i in tqdm.tnrange(n_perms):
    # Permute edges
    perm_edges, _ = xswap.permute_edge_list(perm_edges, allow_self_loops=False, 
                                            allow_antiparallel=True, seed=i)
    perm_mat = analysis.edges_to_matrix(perm_edges)
    feature_mats['prior_empirical'] += perm_mat
    
    # Compute RWR on permuted network
    perm_rwr = analysis.invertible_rwr(perm_mat.toarray(), 0.25)
    feature_mats['mean_rwr'] += perm_rwr
    feature_mats['p_rwr'] += (perm_rwr >= feature_mats['rwr'])
    
    # Compute Jaccard similarity on permuted network
    A2 = perm_mat@perm_mat.T
    perm_jac = A2 / (degree - A2)
    feature_mats['mean_jaccard'] += perm_jac
    feature_mats['p_jaccard'] += (perm_jac >= feature_mats['jaccard'])

# Normalize features to number of permutations
for feature in ['mean_rwr', 'p_rwr', 'mean_jaccard', 'p_jaccard', 'prior_empirical']:
    feature_mats[feature] = feature_mats[feature] / n_perms

In [53]:
source_nodes = sorted(set(edge[0] for edge in connected_edges))
target_nodes = sorted(set(edge[1] for edge in connected_edges))

In [54]:
len(source_nodes), len(target_nodes)

(5386, 7452)

In [ ]:
# Add computed features to the DataFrame
for feature, values in feature_dict.items():
    df[feature] = np.array(v).flatten()

In [44]:
len(set(df['tf']))

232

In [46]:
len(set(edge[0] for edge in edges))

225

In [48]:
edge = (1, 2)

In [49]:
edge in original_edges

False

In [50]:
edge not in original_edges

True

In [51]:
not edge in original_edges


True